In [1]:
from fabric import Connection
from subprocess import Popen, PIPE, CalledProcessError
from dateutil import parser
import copy
import time
import uuid
import os
import io
from invoke import Responder
from IPython.display import display, clear_output

import gce_api

#This can be used to call local shell commands
options = {'stdout': PIPE, 'stderr': PIPE, 'bufsize' : 1, 'universal_newlines' : True, 'shell' : False}
def callPopen(cmd,verbose=True,overwrite=False,additionalDisplay=''):
    with Popen(cmd.split(),**options) as p:
        if verbose and not overwrite:
            for line in p.stdout:
                print(line, end='')
        if verbose and overwrite:
            dq = collections.deque(maxlen=10)
            for line in p.stdout:
                dq.append(line)
                clear_output(wait=True)
                print(additionalDisplay,''.join(list(dq)),sep='\n')
        for line in p.stderr:
            print(line, end='')
        if p.returncode != (0 or None):
            raise CalledProcessError(p.returncode, p.args)

# Setting up ftp server for Mobius models
## Creating virtual machine with dependencies

In [2]:
#Testing connection to google cloud
cloudInfo = {'project': 'nivacatchment',
             'zone': 'europe-west3-a'
             }
gce = gce_api.gce_api('/home/jose-luis/Envs/gce_framework/code/keys/nivacatchment.json',cloudInfo)

info = gce.get('projectInfo')
display('Can now talk to project {}'.format(info['name']))

{}

'Can now talk to project nivacatchment'

In [3]:
#Getting list of debian images that have not been deprecated
debian = copy.deepcopy(cloudInfo)
debian['project'] = 'debian-cloud'
info = gce.get('imagesList', properties=debian)
currentImages = sorted([(i['selfLink'],parser.parse(i['creationTimestamp'])) for i in info['items'] if 'deprecated' not in i],key=lambda x:x[1])
display(currentImages)

{}

[('https://www.googleapis.com/compute/v1/projects/debian-cloud/global/images/debian-9-stretch-v20190618',
  datetime.datetime(2019, 6, 18, 16, 3, 47, 268000, tzinfo=tzoffset(None, -25200)))]

In [4]:
#Creating firewall rule that will allow ftp connections to the vm
display(cloudInfo)
info = gce.get('firewallList',properties=cloudInfo)
firewalls = [i['name'] for i in info['items']]
ftp = {
  "name" : "ftp",  
  "allowed": [
    {
      "IPProtocol": "tcp",
      "ports": [
        "20",
        "21",
        "40110-40210"
      ]
    }
  ],
  "sourceRanges": [
    "151.157.0.0/16",
    "84.213.196.40/32"
  ],
  "targetTags": [
    "ftp"
  ]
}

if 'ftp' in firewalls:
    cloudInfo['firewallName'] = 'ftp'
    info = gce.delete('firewallResource')
    display(info)

#Waiting until the firewall has been deleted
info = gce.get('firewallList')
firewalls = [i['name'] for i in info['items']]

while 'ftp' in firewalls:
    time.sleep(0.5)
    info=gce.get('firewallList')
    firewalls = [i['name'] for i in info['items']]
 
# Actually creating the firewall
info = gce.post('firewallList',json=ftp)
display(info)

{'project': 'nivacatchment', 'zone': 'europe-west3-a'}

{}

{}

{'id': '6293874932084185096',
 'name': 'operation-1563469543003-58df79eba548c-837638da-55d631ac',
 'operationType': 'delete',
 'targetLink': 'https://www.googleapis.com/compute/v1/projects/nivacatchment/global/firewalls/ftp',
 'targetId': '6818240217873193648',
 'status': 'RUNNING',
 'user': 'workshop-key@nivacatchment.iam.gserviceaccount.com',
 'progress': 0,
 'insertTime': '2019-07-18T10:05:43.445-07:00',
 'startTime': '2019-07-18T10:05:43.459-07:00',
 'selfLink': 'https://www.googleapis.com/compute/v1/projects/nivacatchment/global/operations/operation-1563469543003-58df79eba548c-837638da-55d631ac',
 'kind': 'compute#operation'}

{}

...

{}

{}

{}

{'json': {'name': 'ftp',
  'allowed': [{'IPProtocol': 'tcp', 'ports': ['20', '21', '40110-40210']}],
  'sourceRanges': ['151.157.0.0/16', '84.213.196.40/32'],
  'targetTags': ['ftp']}}

{'kind': 'compute#operation',
 'id': '5453170173774993439',
 'name': 'operation-1563469551317-58df79f39310d-c5f81420-3f8c0613',
 'operationType': 'insert',
 'targetLink': 'https://www.googleapis.com/compute/v1/projects/nivacatchment/global/firewalls/ftp',
 'targetId': '7725773164069591071',
 'status': 'RUNNING',
 'user': 'workshop-key@nivacatchment.iam.gserviceaccount.com',
 'progress': 0,
 'insertTime': '2019-07-18T10:05:52.353-07:00',
 'startTime': '2019-07-18T10:05:52.368-07:00',
 'selfLink': 'https://www.googleapis.com/compute/v1/projects/nivacatchment/global/operations/operation-1563469551317-58df79f39310d-c5f81420-3f8c0613'}

In [5]:
#Unique id for the instantiation request
requestId =  str(uuid.uuid4())
display(requestId)

'431b9b3d-743a-4662-8ee6-ff940ca9bbcd'

In [6]:
#Setting VM infoCreating ssh file
gce.properties['instanceType'] = "n1-standard-2"
gce.properties['instanceName'] = "mobiserver"
gce.properties['username'] = "jose-luis"
gce.properties['keyDir'] = './'
gce.properties['keyFile'] = F'{os.path.join(gce.properties["keyDir"],gce.properties["username"])}'
gce.properties['pubKeyFile'] = F'{gce.properties["keyFile"] + ".pub"}'

gce.generateSSHKey()

(b"Generating public/private rsa key pair.\n./jose-luis already exists.\nOverwrite (y/n)? Your identification has been saved in ./jose-luis.\nYour public key has been saved in ./jose-luis.pub.\nThe key fingerprint is:\nSHA256:HrHVP2GdENDiXXn4mhxTWdAtYvn2Nv8MrTJK77qqtlw jose-luis\nThe key's randomart image is:\n+---[RSA 2048]----+\n|           .o+o+*|\n|           o+.+=*|\n|        . o.+o+=o|\n|         + . +=..|\n|        S    oo* |\n|       . .    ++o|\n|        E .   ..+|\n|     ... . .o  +.|\n|     .+o..+=oo. +|\n+----[SHA256]-----+\n", b'')


In [7]:
with open (gce.properties['pubKeyFile'],'r') as f:
    pub = f.read().strip()

In [8]:
#Installation script for the VM
#Startup script
initScript = '''#! /bin/bash
yes | sudo apt-get update
yes | sudo apt-get install ftp lsof bzip2 openssl gcc make pure-ftpd pure-ftpd-common
#yes | curl https://download.pureftpd.org/pub/pure-ftpd/releases/pure-ftpd-1.0.46.tar.bz2 >  pure.tar.bz2
#yes | tar -xf pure.tar.bz2
sudo /usr/sbin/groupadd _pure-ftpd
sudo /usr/sbin/useradd -g _pure-ftpd -d /var/empty -s /etc _pure-ftpd
#cd ./pure-ftpd-1.0.46
#./configure --with-everything
#sudo make install-strip
'''

# '''#! /bin/bash
# yes | sudo apt-get update
# yes | sudo apt-get install ftp pure-ftpd lsof xinetd openssl
# cat <<EOF > /etc/xinetd.d/pure-ftpd
# service ftp
# {
#         socket_type = stream
#         server = /usr/sbin/pure-ftp
#         server_args = -a 408 -c 5 -i -F /root/FTPmessage -O clf:var/log/pureftpd.log -N
#         protocol = tcp
#         user = root
#         wait = no
#         disable = no
#         only_from 0.0.0.0/0
# }
# EOF
# sudo service pure-ftpd stop
# sudo service xinetd start
# '''


machineProperties = {
  "machineType" : "zones/{}/machineTypes/{}".format(gce.properties['zone'],gce.properties['instanceType']),
  "name" : gce.properties['instanceName'],
  "canIpForward": "false",
  "networkInterfaces": [
     {
       "subnetwork": "projects/{}/regions/{}/subnetworks/default".format(gce.properties['project'], '-'.join(gce.properties['zone'].split('-')[:-1]) ),
       "accessConfigs": [
        {
          "kind": "compute#networkInterface",
          "name": "External NAT",
          "type": "ONE_TO_ONE_NAT",
          "networkTier": "PREMIUM"
        }
       ]     
     }
     ],
    
 "tags": {
    "items": [
      "ftp"
    ]
  },
  "disks": [
    {
      "boot": True,
      "autoDelete": True,
      "deviceName": gce.properties['instanceName'],
      "initializeParams": {
        "sourceImage": currentImages[-1][0],
#         "diskType": "projects/{}/zones/{}/diskTypes/pd-standard".format(gce.project,gce.zone),
#         "diskSizeGb": diskSize
      }
    }
  ],
  "metadata": {
    "items": [
      {
       "key": "ssh-keys",
       "value": gce.properties["username"] + ':' + pub
      },
      {
       "key": "startup-script",
       "value": initScript
      },
        
    ]
  } 
}

params = {
    'requestId': requestId,
}

def getNames(info):
    return [i['name'] for i in info['items']]

info = gce.get('instances')
if gce.properties['instanceName'] not in getNames(info):
    info = gce.post('instances',json=machineProperties)    

{}

{'json': {'machineType': 'zones/europe-west3-a/machineTypes/n1-standard-2',
  'name': 'mobiserver',
  'canIpForward': 'false',
  'networkInterfaces': [{'subnetwork': 'projects/nivacatchment/regions/europe-west3/subnetworks/default',
    'accessConfigs': [{'kind': 'compute#networkInterface',
      'name': 'External NAT',
      'type': 'ONE_TO_ONE_NAT',
      'networkTier': 'PREMIUM'}]}],
  'tags': {'items': ['ftp']},
  'disks': [{'boot': True,
    'autoDelete': True,
    'deviceName': 'mobiserver',
    'initializeParams': {'sourceImage': 'https://www.googleapis.com/compute/v1/projects/debian-cloud/global/images/debian-9-stretch-v20190618'}}],
  'metadata': {'items': [{'key': 'ssh-keys',
     'value': 'jose-luis:ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQDmftdop1X5gPThaz5McHS08zVica1Y4+ohyFQGR2d+/WU8zqcpy8k2imjV0MMYIVrcTYhdpLaQ8VV/iB2VMHRjV4Eg4EJtwIqxRvL37DCvn6jB6A1KuD2uX/EeCKKSlAiuHcBr+hMKncB0LfrZLtKD1m/NyKbXM2wzJjeEuouW80e7qPd2ul9u7kIo++7m6eNfG2+ukyPU7XUBQnwekRvcuHaVOPw4UmJFWcUajOeEg1Mf0k3x

In [9]:
#Waiting until the instance is running
info = gce.get('instanceInfo')
display(info['status'])
while (info['status'] != 'RUNNING'):
    time.sleep(.1)
    info  = gce.get('instanceInfo')
    display(info['status'])

gce.properties['ip'] = info['networkInterfaces'][0]['accessConfigs'][0]['natIP']
display(gce.properties['ip'])


{}

'PROVISIONING'

{}

'PROVISIONING'

{}

...

{}

'STAGING'

{}

'RUNNING'

'35.246.138.174'

In [10]:
#Waiting for startup script to complete
params={
    'port': 1,
    'start': 0
}

response=gce.get('serialPort',params=params)
while 'Startup finished in' not in response['contents']:
    time.sleep(5)
    clear_output()
    params['start'] = int(response['next'])
    response=gce.get('serialPort',params=params)
    lines = response['contents'].splitlines()
    display(lines[-10:])
    
clear_output()
lines = response['contents'].splitlines()
display([i for i in lines if "Startup finished in" in i])

['Jul 18 17:06:21 mobiserver systemd[1]: Startup finished in 1.568s (kernel) + 18.455s (userspace) = 20.023s.']

In [11]:
#Testing connection with Fabric
c = Connection(
    host=gce.properties['ip'],
    user=gce.properties['username'],
    connect_kwargs={
        "key_filename": gce.properties['keyFile'],
    },
)

with io.open('output.txt', 'w') as file:
    p = c.run('yes | ls -lah /usr/sbin',pty=True,echo=True,out_stream=file)

if p.stderr != '':
    display("There were some errors: {}".format(p.stderr))
else:
    display("Command output stored in {}".format(file.name))


yes | ls -lah /usr/sbin


'Command output stored in output.txt'

In [12]:
#Adding a system user that will "host" all virtual users
#This is to avoid creating a system user for all the people that are granted access right
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

c.sudo('/usr/sbin/groupadd -f ftpgroup')
#Checking if user exists before adding it
result = c.run('getent passwd ftpuser',warn=True)
if result.stdout != '':
    c.sudo('/usr/sbin/userdel ftpuser')
#Adding user to the group    
c.sudo('/usr/sbin/useradd -g ftpgroup -d /dev/null -s /etc ftpuser')
#Creating a base directory for all users
c.sudo('mkdir /home/ftp')
c.sudo('chmod 0777 /home/')

<Result cmd="sudo -S -p '[sudo] password: ' chmod 0777 /home/" exited=0>

In [13]:
user = 'magnus'
password = 'dummy'
#First checking if user exists
result = c.sudo('''pure-pw list | awk '{{print match($1,"{}") ? "true" : "false"}} ' '''.format(user))
isUser = [i=='true' for i in result.stdout.splitlines()]
#Deleting it if it does
if any(isUser):
    display("Gonna delete {}".format(user))
    c.sudo('pure-pw userdel {}'.format(user))
    c.sudo('pure-pw mkdb')
#Creating user 
enterPass = Responder(
        pattern=r"Password:",
        response='{}\n'.format(password),
    )
confirmPass = Responder(
        pattern=r"Enter it again:",
        response='{}\n'.format(password),
    )

c.sudo('pure-pw useradd {0} -u ftpuser -d /home/ftp/'.format(user),watchers=[enterPass,confirmPass])
c.sudo('pure-pw mkdb')

Unable to open the passwd file: No such file or directory


Password: 
Enter it again: 


<Result cmd="sudo -S -p '[sudo] password: ' pure-pw useradd magnus -u ftpuser -d /home/ftp/" exited=0>

<Result cmd="sudo -S -p '[sudo] password: ' pure-pw mkdb" exited=0>

In [14]:
#Adding puredb (virtual users) as the only (?) authentication method
c.sudo('ln -sf /etc/pure-ftpd/conf/PureDB /etc/pure-ftpd/auth/PureDB')
c.run('echo "yes" | sudo tee /etc/pure-ftpd/conf/ChrootEveryone')
c.run('echo "no"  | sudo tee /etc/pure-ftpd/conf/CreateHomeDir')
#c.run('echo "no"  | sudo tee /etc/pure-ftpd/conf/UnixAuthentication')
c.run('echo "no"  | sudo tee /etc/pure-ftpd/conf/PAMAuthentication')
c.run('echo "no"  | sudo tee /etc/pure-ftpd/conf/NoAnonymous')
c.run('echo "1"   | sudo tee /etc/pure-ftpd/conf/TLS')
c.run('echo "40110 40210" | sudo tee /etc/pure-ftpd/conf/PassivePortRange')
c.run('echo "{}" | sudo tee /etc/pure-ftpd/conf/ForcePassiveIP'.format(gce.properties['ip']))
c.run('mkdir -p /etc/ssl/private')
c.run('sudo openssl dhparam -out /etc/ssl/private/pure-ftpd-dhparams.pem 2048')

#Need to complete the dialog for the generation of the key file
SSLsettings = [ Responder(
        pattern=r"Country Name .*",
        response='NO\n',
                          ),
               Responder(
        pattern=r"State or Province Name .*",
        response='Oslo\n',
                          ),
               Responder(
        pattern=r"Locality Name .*",
        response='Oslo\n',
                          ),
               Responder(
        pattern=r"Organization Name .*",
        response='NIVA\n',
                          ),
               Responder(
        pattern=r"Organizational Unit Name .*",
        response='Catchment Biogeochemistry\n',
                          ),
               Responder(
        pattern=r"Common Name .*",
        response='Jose-Luis\n',
                          ),
               Responder(
        pattern=r"Email Address .*",
        response='jlg@niva.no\n',
                          )
    ]
c.run('sudo openssl req -x509 -nodes -newkey rsa:2048 -sha256 -keyout \
  /etc/ssl/private/pure-ftpd.pem \
  -out /etc/ssl/private/pure-ftpd.pem', pty=True, watchers=SSLsettings)
c.sudo('chmod 600 /etc/ssl/private/pure-ftpd.pem')
c.sudo('chmod 600 /etc/ssl/private/pure-ftpd-dhparams.pem')
c.sudo('service pure-ftpd restart')

<Result cmd="sudo -S -p '[sudo] password: ' ln -sf /etc/pure-ftpd/conf/PureDB /etc/pure-ftpd/auth/PureDB" exited=0>

yes


<Result cmd='echo "yes" | sudo tee /etc/pure-ftpd/conf/ChrootEveryone' exited=0>

no


<Result cmd='echo "no"  | sudo tee /etc/pure-ftpd/conf/CreateHomeDir' exited=0>

...

Generating a RSA private key
.............................................................+++++
.............................+++++
writing new private key to '/etc/ssl/private/pure-ftpd.pem'
-----
You are about to be asked to enter information that will be incorporated
into your certificate request.
What you are about to enter is what is called a Distinguished Name or a DN.
There are quite a few fields but you can leave some blank
For some fields there will be a default value,
If you enter '.', the field will be left blank.
-----
Country Name (2 letter code) [AU]:NO
State or Province Name (full name) [Some-State]:Oslo
Locality Name (eg, city) []:Oslo
Organization Name (eg, company) [Internet Widgits Pty Ltd]:NIVA
Organizational Unit Name (eg, section) []:Catchment Biogeochemistry
Common Name (e.g. server FQDN or YOUR name) []:Jose-Luis
Email Address []:jlg@niva.no


<Result cmd='sudo openssl req -x509 -nodes -newkey rsa:2048 -sha256 -keyout   /etc/ssl/private/pure-ftpd.pem   -out /etc/ssl/private/pure-ftpd.pem' exited=0>

<Result cmd="sudo -S -p '[sudo] password: ' chmod 600 /etc/ssl/private/pure-ftpd.pem" exited=0>

<Result cmd="sudo -S -p '[sudo] password: ' chmod 600 /etc/ssl/private/pure-ftpd-dhparams.pem" exited=0>

<Result cmd="sudo -S -p '[sudo] password: ' service pure-ftpd restart" exited=0>

In [15]:
display(gce.properties['ip'])

'35.246.138.174'

In [17]:
import pyperclip
pyperclip.copy(gce.properties['ip'])

### Setting up the directories for